# Airline on Time Data
## Introduction
Have you ever been stuck in an airport because your flight was delayed or cancelled and wondered if you could have predicted it if you'd had more data? This is your chance to find out. 

The data: The data consists of flight arrival and departure details for all commercial flights within the USA, from October 1987 to April 2008. This is a large dataset: there are nearly 120 million records in total, and takes up 1.6 gigabytes of space compressed and 12 gigab ytes when uncompressed. The data comes originally from RITA where it is described in detail. You can download the data [here](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/HG7NV7).

Variable descriptions: 

S/N | Name | Description 
-- | -- | --
1 | `Year` | 1987-2008 
2 | `Month` | 1-12 
3 | `DayofMonth` | 1-31 
4 | `DayOfWeek` | 1 (Monday) - 7 (Sunday) 
5 | `DepTime` | actual departure time (local, hhmm) 
6 | `CRSDepTime` | scheduled departure time (local, hhmm) 
7 | `ArrTime` | actual arrival time (local, hhmm) 
8 | `CRSArrTime` | scheduled arrival time (local, hhmm) 
9 | `UniqueCarrier` | unique carrier code 
10 | `FlightNum` | flight number 
11 | `TailNum` | plane tail number 
12 | `ActualElapsedTime` | actual elapsed time in minutes 
13 | `CRSElapsedTime` | scheduled elapsed time in minutes 
14 | `AirTime` | the time from the moment an aircraft leaves the surface until it comes into contact with the surface at the next point of landing;,in minutes 
15 | `ArrDelay` | arrival delay, in minutes 
16 | `DepDelay` | departure delay, in minutes 
17 | `Origin` | origin, IATA airport code 
18 | `Dest` | destination, IATA airport code 
19 | `Distance` | distance covered, in miles 
20 | `TaxiIn` | taxi in time, in minutes 
21 | `TaxiOut` | taxi out time in minutes 
22 | `Cancelled` | was the flight cancelled? 
23 | `CancellationCode` | reason for cancellation (A = carrier, B = weather, C = NAS, D = security) 
24 | `Diverted` | 1 = yes, 0 = no 
25 | `CarrierDelay` | delay within air carrier's control, in minutes 
26 | `WeatherDelay` | delay caused by extreme weather conditions, in minutes 
27 | `NASDelay` | delay within the NAS control, in minutes 
28 | `SecurityDelay` | delay due to security checks, breach or faulty security equipment, in minutes 
29 | `LateAircraftDelay` | delay due to the late arrival of the same aircraft at a previous airport, in minutes

* The International Air Transport Association's (IATA) Location Identifier is a unique 3-letter code (also commonly known as IATA code) used in aviation and also in logistics to identify an airport.
* A computer reservation system or a central reservation system (CRS) is a web-based software used by travel agencies and travel management companies to retrieve and conduct transactions related to air travel, hotels, car rental, or other activities. 

In [2]:
# import modules and libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob

%matplotlib inline

Use `glob()` to list all files that match a pattern and sort results.

In [3]:
files = sorted(glob('dataset/*.csv'))
files

['dataset\\1987.csv',
 'dataset\\1988.csv',
 'dataset\\1989.csv',
 'dataset\\1990.csv',
 'dataset\\1991.csv',
 'dataset\\1992.csv',
 'dataset\\1993.csv',
 'dataset\\1994.csv',
 'dataset\\1995.csv',
 'dataset\\1996.csv',
 'dataset\\1997.csv',
 'dataset\\1998.csv',
 'dataset\\1999.csv',
 'dataset\\2000.csv',
 'dataset\\2001.csv',
 'dataset\\2002.csv',
 'dataset\\2003.csv',
 'dataset\\2004.csv',
 'dataset\\2005.csv',
 'dataset\\2006.csv',
 'dataset\\2007.csv',
 'dataset\\2008.csv']

Use a generator expression to read the files and `concat()` to append dataframes by row.

In [27]:
airlineTime_df = pd.concat((pd.read_csv(file, low_memory=False) for file in files), ignore_index=True)

ParserError: Error tokenizing data. C error: out of memory

In [20]:
df = pd.read_csv(files[1])

In [25]:
np.arange(1987,2009)

array([1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
       1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008])